# 📊 Notebook 01: Data Pipeline & Preprocessing

## AADHAAR INTELLIGENCE SYSTEM - UIDAI Hackathon 2025-26

---

### Objective
Extract, transform, and load the **3 Real UIDAI Datasets** for behavioral analytics:
1. **Enrollment Data** - New Aadhaar registrations by age group (0-5, 5-17, 18+)
2. **Demographic Update Data** - Address/Name updates by age group
3. **Biometric Update Data** - Fingerprint/Iris updates by age group

### Dataset Structure
- **Enrolment**: `date, state, district, pincode, age_0_5, age_5_17, age_18_greater`
- **Demographic**: `date, state, district, pincode, demo_age_5_17, demo_age_17_`
- **Biometric**: `date, state, district, pincode, bio_age_5_17, bio_age_17_`

### Output
- Cleaned, merged master dataset
- Data quality report
- Export for downstream analysis

In [ ]:
# ============================================
# CELL 1: Import Libraries
# ============================================

import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("✅ Libraries imported successfully")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"🐍 Pandas version: {pd.__version__}")

In [ ]:
# ============================================
# CELL 2: Configuration & Paths
# ============================================

# Define paths
DATA_DIR = '../data/'
OUTPUT_DIR = '../outputs/'

# Dataset folder paths (extracted from zip files)
DATASET_PATHS = {
    'enrolment': f"{DATA_DIR}enrolment/",
    'demographic': f"{DATA_DIR}demographic/",
    'biometric': f"{DATA_DIR}biometric/"
}

# Create output directory if not exists
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/charts", exist_ok=True)

print("📁 Configuration set:")
print(f"   Data Directory: {DATA_DIR}")
print(f"   Output Directory: {OUTPUT_DIR}")
print(f"\n📂 Dataset Paths:")
for name, path in DATASET_PATHS.items():
    print(f"   • {name}: {path}")

In [ ]:
# ============================================
# CELL 3: Data Loading Function
# ============================================

def load_all_csvs(folder_path, dataset_name):
    """
    Load and concatenate all CSV files from a folder
    
    Parameters:
    -----------
    folder_path : str - Path to folder containing CSVs
    dataset_name : str - Name for logging
    
    Returns:
    --------
    pd.DataFrame - Concatenated dataset
    """
    print(f"\n📦 Loading: {dataset_name.upper()}")
    print(f"   Path: {folder_path}")
    
    # Find all CSV files recursively
    all_files = glob.glob(os.path.join(folder_path, "**/*.csv"), recursive=True)
    
    if not all_files:
        print(f"   ⚠️ No CSV files found in {folder_path}")
        return None
    
    print(f"   📄 Found {len(all_files)} CSV files")
    
    # Load and concatenate all CSVs
    dfs = []
    total_rows = 0
    for file in all_files:
        df = pd.read_csv(file)
        dfs.append(df)
        total_rows += len(df)
        print(f"      ✓ {os.path.basename(file)}: {len(df):,} rows")
    
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"   ✅ Total loaded: {len(combined_df):,} records")
    
    return combined_df

print("✅ Data loading function ready")

In [ ]:
# ============================================
# CELL 4: Load All 3 UIDAI Datasets
# ============================================

print("="*60)
print("🔄 LOADING REAL UIDAI DATASETS")
print("="*60)

# Load Enrolment Data
df_enrolment = load_all_csvs(DATASET_PATHS['enrolment'], 'enrolment')

# Load Demographic Update Data  
df_demographic = load_all_csvs(DATASET_PATHS['demographic'], 'demographic')

# Load Biometric Update Data
df_biometric = load_all_csvs(DATASET_PATHS['biometric'], 'biometric')

print("\n" + "="*60)
print("✅ ALL DATASETS LOADED SUCCESSFULLY")
print("="*60)
print(f"\n📈 Dataset Summary:")
print(f"   Enrolment Records: {len(df_enrolment):,}")
print(f"   Demographic Records: {len(df_demographic):,}")
print(f"   Biometric Records: {len(df_biometric):,}")

In [ ]:
# ============================================
# CELL 5: Data Exploration
# ============================================

print("\n🔍 DATA EXPLORATION")
print("="*60)

# Display column info for each dataset
print("\n📋 ENROLMENT DATA:")
print(f"   Columns: {df_enrolment.columns.tolist()}")
print(f"   Shape: {df_enrolment.shape}")
display(df_enrolment.head(3))

print("\n📋 DEMOGRAPHIC UPDATE DATA:")
print(f"   Columns: {df_demographic.columns.tolist()}")
print(f"   Shape: {df_demographic.shape}")
display(df_demographic.head(3))

print("\n📋 BIOMETRIC UPDATE DATA:")
print(f"   Columns: {df_biometric.columns.tolist()}")
print(f"   Shape: {df_biometric.shape}")
display(df_biometric.head(3))

In [ ]:
# ============================================
# CELL 6: Data Preprocessing
# ============================================

print("\n🧹 DATA PREPROCESSING")
print("="*60)

# Convert date columns
df_enrolment['date'] = pd.to_datetime(df_enrolment['date'], format='%d-%m-%Y')
df_demographic['date'] = pd.to_datetime(df_demographic['date'], format='%d-%m-%Y')
df_biometric['date'] = pd.to_datetime(df_biometric['date'], format='%d-%m-%Y')

# Calculate total enrollments per record
df_enrolment['total_enrolments'] = df_enrolment['age_0_5'] + df_enrolment['age_5_17'] + df_enrolment['age_18_greater']

# Fix column names (strip trailing characters)
df_demographic.columns = df_demographic.columns.str.strip('_')
df_biometric.columns = df_biometric.columns.str.strip('_')

# Rename columns for consistency
if 'demo_age_17' in df_demographic.columns:
    df_demographic.rename(columns={'demo_age_17': 'demo_age_18_greater'}, inplace=True)
if 'bio_age_17' in df_biometric.columns:
    df_biometric.rename(columns={'bio_age_17': 'bio_age_18_greater'}, inplace=True)

# Calculate totals for demographic and biometric
df_demographic['total_demo_updates'] = df_demographic.filter(like='demo_age').sum(axis=1)
df_biometric['total_bio_updates'] = df_biometric.filter(like='bio_age').sum(axis=1)

# Add derived date columns
for df in [df_enrolment, df_demographic, df_biometric]:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    df['day_of_week'] = df['date'].dt.dayofweek

print("✅ Date columns converted to datetime")
print(f"   Enrolment date range: {df_enrolment['date'].min()} to {df_enrolment['date'].max()}")
print(f"   Demographic date range: {df_demographic['date'].min()} to {df_demographic['date'].max()}")
print(f"   Biometric date range: {df_biometric['date'].min()} to {df_biometric['date'].max()}")

In [ ]:
# ============================================
# CELL 7: Data Quality Report
# ============================================

def data_quality_report(df, name):
    """Generate comprehensive data quality report"""
    print(f"\n{'='*60}")
    print(f"📋 DATA QUALITY REPORT: {name.upper()}")
    print(f"{'='*60}")
    
    print(f"\n📊 BASIC STATISTICS:")
    print(f"   Total Records: {len(df):,}")
    print(f"   Total Columns: {len(df.columns)}")
    print(f"   Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    print(f"\n📝 COLUMN INFORMATION:")
    for col in df.columns:
        null_pct = df[col].isnull().sum() / len(df) * 100
        unique = df[col].nunique()
        dtype = df[col].dtype
        print(f"   • {col}: {dtype} | Unique: {unique:,} | Null: {null_pct:.1f}%")
    
    return {
        'name': name,
        'records': len(df),
        'columns': len(df.columns),
        'null_pct': df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100
    }

# Generate reports for all datasets
quality_reports = []
quality_reports.append(data_quality_report(df_enrolment, 'Enrollment'))
quality_reports.append(data_quality_report(df_demographic, 'Demographic'))
quality_reports.append(data_quality_report(df_biometric, 'Biometric'))

In [ ]:
# ============================================
# CELL 8: Create Pincode-Level Master Dataset
# ============================================

print("\n📍 CREATING PINCODE-LEVEL MASTER DATASET")
print("="*60)

# Aggregate enrollment by pincode
enrolment_by_pincode = df_enrolment.groupby(['state', 'district', 'pincode']).agg({
    'age_0_5': 'sum',
    'age_5_17': 'sum',
    'age_18_greater': 'sum',
    'total_enrolments': 'sum',
    'date': 'count'
}).reset_index()
enrolment_by_pincode.rename(columns={'date': 'enrolment_days'}, inplace=True)

# Aggregate demographic updates by pincode
demo_by_pincode = df_demographic.groupby(['state', 'district', 'pincode']).agg({
    'total_demo_updates': 'sum'
}).reset_index()

# Aggregate biometric updates by pincode
bio_by_pincode = df_biometric.groupby(['state', 'district', 'pincode']).agg({
    'total_bio_updates': 'sum'
}).reset_index()

# Merge all datasets
master_pincode = enrolment_by_pincode.merge(
    demo_by_pincode[['pincode', 'total_demo_updates']], 
    on='pincode', 
    how='left'
).merge(
    bio_by_pincode[['pincode', 'total_bio_updates']], 
    on='pincode', 
    how='left'
)

# Fill NaN values
master_pincode['total_demo_updates'] = master_pincode['total_demo_updates'].fillna(0)
master_pincode['total_bio_updates'] = master_pincode['total_bio_updates'].fillna(0)

# Calculate total activity
master_pincode['total_activity'] = (
    master_pincode['total_enrolments'] + 
    master_pincode['total_demo_updates'] + 
    master_pincode['total_bio_updates']
)

# Calculate daily rate
master_pincode['daily_enrolment_rate'] = master_pincode['total_enrolments'] / master_pincode['enrolment_days']

print(f"\n📊 Master Pincode Dataset Created:")
print(f"   Total Unique Pincodes: {len(master_pincode):,}")
print(f"   Total States/UTs: {master_pincode['state'].nunique()}")
print(f"   Total Enrolments: {master_pincode['total_enrolments'].sum():,}")
print(f"   Total Demo Updates: {master_pincode['total_demo_updates'].sum():,.0f}")
print(f"   Total Bio Updates: {master_pincode['total_bio_updates'].sum():,.0f}")

display(master_pincode.head(10))

In [ ]:
# ============================================
# CELL 9: Data Summary Visualization
# ============================================

# Create summary visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Enrollments by State (Top 10)', 
        'Age Group Distribution',
        'Monthly Enrollment Trend',
        'Daily Activity Pattern'
    ),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# Plot 1: Enrollments by State
state_enrol = master_pincode.groupby('state')['total_enrolments'].sum().sort_values(ascending=True).tail(10)
fig.add_trace(
    go.Bar(x=state_enrol.values, y=state_enrol.index, orientation='h', 
           marker_color='#FF6B35', name='Enrollments'),
    row=1, col=1
)

# Plot 2: Age Group Distribution
age_totals = {
    '0-5 years': df_enrolment['age_0_5'].sum(),
    '5-17 years': df_enrolment['age_5_17'].sum(),
    '18+ years': df_enrolment['age_18_greater'].sum()
}
fig.add_trace(
    go.Pie(labels=list(age_totals.keys()), values=list(age_totals.values()),
           marker_colors=['#1B998B', '#F77F00', '#D62828']),
    row=1, col=2
)

# Plot 3: Monthly Trend
monthly = df_enrolment.groupby(['year', 'month'])['total_enrolments'].sum().reset_index()
monthly['date'] = pd.to_datetime(monthly[['year', 'month']].assign(day=1))
fig.add_trace(
    go.Scatter(x=monthly['date'], y=monthly['total_enrolments'], mode='lines+markers',
               line_color='#1B998B', name='Trend'),
    row=2, col=1
)

# Plot 4: Daily Activity Pattern
daily_pattern = df_enrolment.groupby('day_of_week')['total_enrolments'].sum()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
fig.add_trace(
    go.Bar(x=days, y=daily_pattern.values, marker_color='#004E89', name='Daily'),
    row=2, col=2
)

fig.update_layout(
    title_text='<b>AADHAAR DATA PIPELINE SUMMARY</b>',
    title_x=0.5,
    height=700,
    showlegend=False,
    template='plotly_white'
)

# Save figure
fig.write_html(f"{OUTPUT_DIR}/charts/01_data_pipeline_summary.html")
print("📊 Chart saved to outputs/charts/01_data_pipeline_summary.html")

In [ ]:
# ============================================
# CELL 10: Export Processed Data
# ============================================

print("\n💾 EXPORTING PROCESSED DATASETS")
print("="*60)

# Save cleaned datasets
df_enrolment.to_csv(f"{OUTPUT_DIR}/enrolment_cleaned.csv", index=False)
df_demographic.to_csv(f"{OUTPUT_DIR}/demographic_cleaned.csv", index=False)
df_biometric.to_csv(f"{OUTPUT_DIR}/biometric_cleaned.csv", index=False)
master_pincode.to_csv(f"{OUTPUT_DIR}/master_pincode.csv", index=False)

print(f"\n✅ Exported files:")
print(f"   • enrolment_cleaned.csv ({len(df_enrolment):,} records)")
print(f"   • demographic_cleaned.csv ({len(df_demographic):,} records)")
print(f"   • biometric_cleaned.csv ({len(df_biometric):,} records)")
print(f"   • master_pincode.csv ({len(master_pincode):,} pincodes)")

In [ ]:
# ============================================
# CELL 11: Pipeline Metrics Summary
# ============================================

print("\n" + "="*60)
print("📊 FINAL PIPELINE METRICS")
print("="*60)

total_records = len(df_enrolment) + len(df_demographic) + len(df_biometric)

metrics = {
    'Total Records Processed': f"{total_records:,}",
    'Unique Pincodes': f"{len(master_pincode):,}",
    'Total States/UTs': f"{master_pincode['state'].nunique()}",
    'Total Enrollments': f"{master_pincode['total_enrolments'].sum():,}",
    'Total Demographic Updates': f"{master_pincode['total_demo_updates'].sum():,.0f}",
    'Total Biometric Updates': f"{master_pincode['total_bio_updates'].sum():,.0f}",
    'Age 0-5 Enrollments': f"{df_enrolment['age_0_5'].sum():,}",
    'Age 5-17 Enrollments': f"{df_enrolment['age_5_17'].sum():,}",
    'Age 18+ Enrollments': f"{df_enrolment['age_18_greater'].sum():,}",
    'Date Range': f"{df_enrolment['date'].min().date()} to {df_enrolment['date'].max().date()}"
}

for key, value in metrics.items():
    print(f"   • {key}: {value}")

print("\n" + "="*60)
print("✅ NOTEBOOK 01 COMPLETE - Proceed to 02_life_events.ipynb")
print("="*60)